In [1]:
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch

from diffusion_edf.graph_parser import RadiusBipartite
from diffusion_edf.gnn_data import FeaturedPoints, GraphEdge

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
device = 'cuda:0'
irreps = "1x0e+1x1e+1x2e"

In [3]:
graph_parser = RadiusBipartite(
    r=2.,
    length_enc_type='SinusoidalPositionEmbeddings',
    length_enc_kwarg={'dim': 128, 'n': 10000},
    sh_irreps=irreps,
).to(device)

graph_parser = torch.jit.script(graph_parser)

####### Figure #######
# out = graph_parser.length_enc(torch.linspace(0,graph_parser.r, 300))
# plt.imshow(out.permute(1,0))

In [4]:
src_pcd = FeaturedPoints(x=torch.tensor([[0., 0., 0.],
                                         [0., 0., 1.],
                                         [0., 1., 0.]], device=device), f=torch.randn(3,3, device=device), b=torch.zeros(3, device=device, dtype=torch.long))
dst_pcd = FeaturedPoints(x=torch.tensor([[0., 0., 0.15],
                                         [0., 0., 1.9]], device=device), f=torch.randn(2,3, device=device), b=torch.zeros(2, device=device, dtype=torch.long))

In [5]:
graph_edge: GraphEdge = graph_parser(src_pcd, dst_pcd)

In [6]:
torch.stack([graph_edge.edge_src, graph_edge.edge_dst])

tensor([[0, 1, 2, 0, 1],
        [0, 0, 0, 1, 1]], device='cuda:0')

In [7]:
graph_edge.edge_scalars.shape

torch.Size([5, 128])

In [8]:
print(f"edge length: {graph_edge.edge_length}\n")
print(f"edge length rel: {graph_edge.edge_length / graph_parser.r}\n")
print(f"scalar cutoff: {graph_edge.edge_weight_scalar}\n")
print(f"vector cutoff: {graph_edge.edge_weight_nonscalar}\n")

edge length: tensor([0.1500, 0.8500, 1.0112, 1.9000, 0.9000], device='cuda:0')

edge length rel: tensor([0.0750, 0.4250, 0.5056, 0.9500, 0.4500], device='cuda:0')

scalar cutoff: tensor([1.0000, 1.0000, 1.0000, 0.1972, 1.0000], device='cuda:0')

vector cutoff: tensor([0.0508, 1.0000, 1.0000, 0.1972, 1.0000], device='cuda:0')



In [9]:
print(f"scalar norm: {graph_edge.edge_attr[...,0]}\n")
print(f"spin-1 norm: {graph_edge.edge_attr[...,1:4].norm(dim=-1)}\n")
print(f"spin-1 norm: {graph_edge.edge_attr[...,4:9].norm(dim=-1)}\n")

scalar norm: tensor([1.0000, 1.0000, 1.0000, 0.1972, 1.0000], device='cuda:0')

spin-1 norm: tensor([0.0880, 1.7321, 1.7321, 0.3415, 1.7321], device='cuda:0')

spin-1 norm: tensor([0.1136, 2.2361, 2.2361, 0.4409, 2.2361], device='cuda:0')



In [10]:
####### Warm Up #######
for _ in range(10):
    src_pcd = FeaturedPoints(x=torch.randn(1000,3, device=device), f=torch.randn(1000,3, device=device), b=torch.zeros(1000, device=device, dtype=torch.long))
    dst_pcd = FeaturedPoints(x=torch.randn(1000,3, device=device) + 2., f=torch.randn(1000,3, device=device), b=torch.zeros(1000, device=device, dtype=torch.long))
    graph_edge: GraphEdge = graph_parser(src_pcd, dst_pcd)

In [11]:
src_pcd = FeaturedPoints(x=torch.randn(1000,3, device=device), f=torch.randn(1000,3, device=device), b=torch.zeros(1000, device=device, dtype=torch.long))
dst_pcd = FeaturedPoints(x=torch.randn(1000,3, device=device) + 2., f=torch.randn(1000,3, device=device), b=torch.zeros(1000, device=device, dtype=torch.long))

for _ in tqdm(range(5000)):
    graph_edge: GraphEdge = graph_parser(src_pcd, dst_pcd)

100%|██████████| 5000/5000 [00:03<00:00, 1503.27it/s]


In [12]:
len(graph_edge.edge_src) # Graph size does not make big difference

52875

In [26]:
src_pcd = FeaturedPoints(x=torch.randn(1000,3, device=device), f=torch.randn(1000,3, device=device), b=torch.zeros(1000, device=device, dtype=torch.long))
dst_pcd = FeaturedPoints(x=torch.randn(1000,3, device=device) + 4., f=torch.randn(1000,3, device=device), b=torch.zeros(1000, device=device, dtype=torch.long))

for _ in tqdm(range(5000)):
    graph_edge: GraphEdge = graph_parser(src_pcd, dst_pcd)

100%|██████████| 5000/5000 [00:03<00:00, 1559.53it/s]


In [27]:
len(graph_edge.edge_src) # Graph size does not make big difference

30

In [28]:
src_pcd = FeaturedPoints(x=torch.randn(1000,3, device=device), f=torch.randn(1000,3, device=device), b=torch.zeros(1000, device=device, dtype=torch.long))
dst_pcd = FeaturedPoints(x=torch.randn(1000,3, device=device) + 4., f=torch.randn(1000,3, device=device), b=torch.zeros(1000, device=device, dtype=torch.long))

for _ in tqdm(range(5000)):
    graph_edge: GraphEdge = graph_parser(src_pcd, dst_pcd, max_neighbors=20)

100%|██████████| 5000/5000 [00:02<00:00, 1757.30it/s]


In [29]:
len(graph_edge.edge_src) # smaller max_neighbor => slightly faster (c.f. max_neighbors = 1000 is default)

31